[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/Video-Infinity-jupyter/blob/main/Video_Infinity_jupyter.ipynb)

In [ ]:
%cd /content
!git clone https://github.com/Yuanshi9815/Video-Infinity
%cd /content/Video-Infinity

!pip install xformers==0.0.26.post1 diffusers==0.27.2 transformers==4.39.3 imageio==2.34.0 imageio-ffmpeg==0.4.9 accelerate==0.29.3

# %cd /content/Video-Infinity
# !python inference.py --config examples/single_gpu.json

import torch
from src.video_crafter import VideoCrafterPipeline
from src.tools import DistController
from src.video_infinity.wrapper import DistWrapper

config = {
    "dtype": "torch.float16",
    "devices": [0,1,2,3],
    "seed": 123,
    "master_port": 29516,
    "base_path": "./exp",
    "pipe_configs":{
        "prompts": [
            "A beagle wearing diving goggles swimming in the ocean while the camera is moving, coral reefs in the background."
        ],
        "steps": 30,
        "guidance_scale": 12,
        "fps": 24,
        "num_frames": 24,
        "height": 320,
        "width": 512,  
        "export_fps": 8,
        "file_name": None
    },
    "plugin_configs":{
        "attn":{
            "padding": 8,
            "top_k": 16,
            "top_k_chunk_size": 24,
            "attn_scale": 1.0,
            "token_num_scale": False,
            "dynamic_scale": True,
            "local_phase": {
                "t": 800, 
                "local_biase": 10,
                "global_biase": 0
            },
            "global_phase": {
                "t": 800,
                "local_biase": 0,
                "global_biase": 10
            }
        },
        "conv_3d": {
            "padding": 1
        }, 
        "conv_layer": {}
    }
}

with torch.inference_mode():
    pipe = VideoCrafterPipeline.from_pretrained('vdo/videocrafterv2_diffusers', torch_dtype=torch.float16)
    pipe.enable_model_cpu_offload(gpu_id=0)
    pipe.enable_vae_slicing()
    pipe_configs=config['pipe_configs']
    plugin_configs=config['plugin_configs']
    prompt = pipe_configs["prompts"][0]
    dist_controller = DistController(0, 1, config)
    dist_pipe = DistWrapper(pipe, dist_controller, config)

In [ ]:
config["pipe_configs"]["prompts"][0] = "A majestic octopus gliding gracefully through crystal-clear azure waters. Its tentacles undulate rhythmically as it navigates a vibrant coral reef. Shafts of sunlight pierce the depths, illuminating the octopus's iridescent skin and creating mesmerizing patterns on the ocean floor. Schools of colorful tropical fish dart around the octopus, completing this captivating underwater scene."
config["pipe_configs"]["num_frames"] = 48

with torch.inference_mode():
    dist_pipe.inference(
        prompt,
        config,
        pipe_configs,
        plugin_configs,
        additional_info={
            "full_config": config,
        }
    )